<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/OSI-SAF_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1b_OSI_SAF_SST_data_access_IFREMER_Opensearch.ipynb" target="_blank">Accessing Metop SST granule data through IFREMER opensearch queries >></a>

<font color="#138D75">**EUMETSAT OSI SAF Training Service**</font> <br>
**Copyright:** 2025 EUMETSAT <br>
**License:** MIT

<html>
  <div style="width:100%">
    <div style="float:left"><a href="https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.eumetsat.int%2Feumetlab%2Foceans%2Focean-training%2Fsensors%2Flearn-osi-saf-sst/HEAD?labpath=1_OSI_SAF_sst_introductory%2F1_1a_OSI_SAF_SST_data_access_IFREMER_FTP.ipynb"><img src="https://mybinder.org/badge_logo.svg" alt="Open in Binder"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
  </div>
</html>

<div class="alert alert-block alert-success">
<h3>Learn OSI SAF sea surface temperature: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **<a href="https://osi-saf.eumetsat.int/register" target="_blank">A EUMETSAT OSI SAF account</a>** if you are using or plan to use the EUMETSAT OSI SAF data.

There are no prerequisite notebooks for this module.
</div>
<hr>

# 1.1a Accessing OSI SAF SST products through the IFREMER FTP server
### Data used 

| Dataset | EUMETSAT collection ID | OSI SAF website description | OSI SAF identifier |
|:-----------------:|:-----------------:|:-----------------:|:-----------------:|
| Global Metop Sea Surface Temperature | EO:EUM:DAT:METOP:GLB-SST-NC | <a href="https://osi-saf.eumetsat.int/products/osi-201-b" target="_blank">Description</a>| OSI-201-b |
| North Atlantic Regional Sea Surface Temperature | EO:EUM:DAT:MULT:OSSTNARN | <a href="https://osi-saf.eumetsat.int/products/osi-202-c" target="_blank">Description</a>| OSI-202-c |
| Meteosat Sea Surface Temperature | EO:EUM:DAT:METEOSAT:OSIHSST-NC | <a href="https://osi-saf.eumetsat.int/products/osi-206-a" target="_blank">Description</a>| OSI-206-a | 
| GOES-East Sea Surface Temperature | EO:EUM:DAT:GOES:OSIHSST-NCC | <a href="https://osi-saf.eumetsat.int/products/osi-207-b" target="_blank">Description</a>| OSI-207-b |
| MSG/SEVIRI Sea Surface Temperature data record | EO:EUM:DAT:MSG:OSI-250 | <a href="https://osi-saf.eumetsat.int/products/osi-250" target="_blank">Description</a>| OSI-250 | 
| Meteosat Indian Ocean Sea Surface Temperature | - | <a href="https://osi-saf.eumetsat.int/products/osi-io-sst" target="_blank">Description</a>| OSI-IO-sst |


### Learning outcomes

At the end of this notebook you will know;
* How to download different OSI SAF sea surface temperature (SST) products using FTP access

### Outline

OSI SAF SST data are available in different distributions. In this repository, we explore three possibilities to access the data. In this first notebook, we'll explore the way to download the data directly from the SST Low and Middle Latitude FTP server, hosted by <a href="https://www.ifremer.fr/en">Ifremer</a>.

<div class="alert alert-info" role="alert">

## <a id='TOC-TOP'></a>Contents

</div>
    
 1. [Registering on OSI-SAF](#section1)
 1. [Creating our workspace](#section2)
 1. [Setting login information](#section3)
 1. [Downloading via FTP requests](#section4)

<hr>

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Registering on OSI-SAF
[Back to top](#TOC-TOP)

</div>

In this notebook, we will be using data from the OSI-SAF products. Before to start accessing data we produce, **<a href="https://osi-saf.eumetsat.int/register" target="_blank">sign up on our website</a>**.

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Creating the workspace
[Back to top](#TOC-TOP)

</div>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [1]:
# library imports 
import os           # a library that allows to access to basic operating system commands like making directories
import json         # a library that helps with JSON format files
import ftplib       # a library that supports FTP protocol use
import warnings     # a library that controls python warnings
import datetime     # a library that controls dates and hours
warnings.filterwarnings('ignore')

Next we will create a download directory to store the products we will download in this notebook.

In [2]:
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section3'></a>3. Setting login information
[Back to top](#TOC-TOP)

</div>
We will access OSI SAF data from the OSI SAF Lower-Mid-Latitude Centre FTP server.

In order to allow us to download data, we need to provide our credentials. We can do this in two ways; either by creating a file called `.eumetsat_osi_saf_sst_credentials` in our home directory (*option 1 - recommended*) or by supplying our credentials directly in this script (*option 2*). 

#### Option 1: creating  `.eumetsat_osi_saf_sst_credentials` in our home directory

For most computer systems the home directory can be found at the path \user\username, /users/username, or /home/username depending on your operating system.

In this file we need to add the following information exactly as follows;

```
{
"username": "anonymous",
"password": "anonymous"
}
```

Make sure to save the file without any kind of extension.

Once you have done this, you can read in your credentials using the commands in the following cell. These will be used to generate a time-limited token, which will refresh itself when it expires.


Reading credential information

In [3]:
# read credentials
with open(os.path.join(os.path.expanduser("~"),'.eumetsat_osi_saf_sst_credentials')) as json_file:
  credentials = json.load(json_file)

Loading credentials

In [4]:
username = credentials['username'] 
password = credentials['password']

#### Option 2: provide credentials directly
You can provide your credentials directly as follows; by uncommenting the following lines.

In [5]:
# username = <provided_username>
# password = <provided_password>

You must replace <provided_username> and <provided_password> with the information provided to you after you have registered to the OSI SAF website and mentionned your interest in SST data.

Once you have created an EUMETSAT OSI SAF account, the generic password and credentials to access OSI SAF Lower-Mid-Latitude  server should have been sent to you via email.

Note: this method is convenient in the short term, but is not really recommended as you have to put your provided username and password in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.

<div class="alert alert-info" role="alert">

## <a id='section4'></a>4. Launch FTP requests
[Back to top](#TOC-TOP)

</div>

In this section, we will download different SST products from OSI-SAF. Each will be used in a different mapping section later on.

### Global Metop Sea Surface Temperature :
We will start by downloading **level 3** data from the <a href="https://osi-saf.eumetsat.int/products/osi-201-b">Global Metop Sea Surface temperature products</a>. This product is an aggregate of 12h of SST measurement done with **AVHRR** aboard the **Metop-B** satellite.<br>

Let's start by creating a specific folder to hold this product.

In [6]:
download_dir = os.path.join(os.getcwd(), "products", "L3_Global_Metop")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [7]:
# Select date
Year, Month, Day = 2025, 7, 15

In [8]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/global/glob_avhrr_metop_b/' + str(Year) + '/' + day_of_year + '/'
filename = str(Year) + Month_str + Day_str + \
        '000000-OSISAF-L3C_GHRSST-SSTsubskin-AVHRR_SST_METOP_B_GLB-sstglb_metop01_'+ \
        str(Year) + Month_str + Day_str + '_000000-v02.0-fv01.0.nc'

In [9]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'L3_Global_Metop', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

###  North Atlantic Regional Sea Surface Temperature:
We will then download another **Level 3** product that offers data at higher spatial and temporal definition over the <a href="https://osi-saf.eumetsat.int/products/osi-202-c">North Atlantic Region (NAR)</a>. It is based on the radiometer aboard two sun-synchronous polar orbiting satellites having complementary orbits (**NOAA-20/VIIRS** and **Metop-B/AVHRR**). It consequently allows to have 4 different SST measurements per day.<br>
Let's create a specific folder to hold the NAR Sea Surface Temperature products.

In [10]:
download_dir = os.path.join(os.getcwd(), "products", "L3_NAR")
os.makedirs(download_dir, exist_ok=True)

#### Noaa 20 NAR data
Let's start by creating a specific folder to hold the data from NOAA-20.

In [11]:
download_dir = os.path.join(os.getcwd(), "products", "L3_NAR", "NOAA-20")
os.makedirs(download_dir, exist_ok=True)

We set the download path where the datas are and we will chose a file in this path.

In [12]:
# Select date for NAR products
Year, Month, Day = 2025, 6, 25

We will set the filename to download the NOAA-20 file timestamped at 01:00UTC (01:00 UTC and 12:00UTC available).

In [13]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/north_atlantic/nar_viirs_noaa_20/' + str(Year) + '/' + str(day_of_year)
filename = str(Year) + Month_str + Day_str + \
        '010000-OSISAF-L3C_GHRSST-SSTsubskin-VIIRS_SST_NOAA20_NAR-sstnar_noaa20_' + \
        str(Year) + Month_str + Day_str + '_010000-v02.0-fv01.0.nc'

In [14]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
file = ftp.nlst()
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'L3_NAR', 'NOAA-20', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

#### Metop B NAR data
Let's start by creating a specific folder to hold the data from Metop-B.

In [15]:
download_dir = os.path.join(os.getcwd(), "products", "L3_NAR", "Metop-B")
os.makedirs(download_dir, exist_ok=True)

We set the dowload path to reach the data are and we will choose a file. As we won't change it, the date taken into account will be the last defined (the one we used for NOAA-20 data). The timestamp we select here below is set for the 10:00UTC file (10:00UTC and 20:00UTC available).

In [16]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/north_atlantic/nar_avhrr_metop_b/' + str(Year) + '/' + day_of_year
filename = str(Year) + Month_str + Day_str + \
        '100000-OSISAF-L3C_GHRSST-SSTsubskin-AVHRR_SST_METOP_B_NAR-sstnar_metop01_' + \
        str(Year) + Month_str + Day_str +'_100000-v02.0-fv01.0.nc'

In [17]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'L3_NAR', 'Metop-B', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Meteosat Sea Surface Temperature :
Let's now download the **level 3** SST based on the geostationnary satellite <a href="https://osi-saf.eumetsat.int/products/osi-206-b">Meteosat</a> operating at 0° of longitude. We will here download data based on the new **Meteosat-12** (formerly MTG-I1). The **FCI** imager aboard the satellite takes a measurement every 10min. This product is an aggregate of the measurement over one hour.<br>

Let's start by creating a specific folder to hold the Meteosat data.

In [18]:
download_dir = os.path.join(os.getcwd(), "products", "L3_Meteosat_0°")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [19]:
# Select date
Year, Month, Day, Hour = 2025, 7, 15, 10

In [20]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)
Hour_str = "{:02d}".format(Hour)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/east_atlantic_west_indian/mtg/' + str(Year) + '/' + day_of_year
filename = str(Year) + Month_str + Day_str + Hour_str + \
        '0000-OSISAF-L3C_GHRSST-SSTsubskin-Meteosat12_FCI-v02.1-fv01.0.nc'

In [21]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'L3_Meteosat_0°', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### GOES-East Sea Surface Temperature
In order to cover all of the atlantic low and middle latitude, OSI SAF provides SST from based on the NOAA geostationnary satellite operating on <a href="https://osi-saf.eumetsat.int/products/osi-207-c">GOES-East postition</a>. As for FCI, the imager ABI aboard GOES-19 takes a measurement every 10min. Our level 3 SST is then an aggregate of the measurements done over one hour.<br>
<br>
Let's start by creating a specific folder to hold the GOES-East data.

In [22]:
download_dir = os.path.join(os.getcwd(), "products", "L3_GOES-E")
os.makedirs(download_dir, exist_ok=True)

We set the dowload path where the datas are and we will chose a file in this path.

In [23]:
# Select date
Year, Month, Day, Hour = 2025, 7, 15, 10

In [24]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)
Hour_str = "{:02d}".format(Hour)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/west_atlantic_east_pacific/goes/'+ str(Year) + '/' + day_of_year
filename = str(Year) + Month_str + Day_str + Hour_str + \
        '0000-OSISAF-L3C_GHRSST-SSTsubskin-GOES19_ABI-v02.1-fv01.0.nc'
print(filename)

20250715100000-OSISAF-L3C_GHRSST-SSTsubskin-GOES19_ABI-v02.1-fv01.0.nc


In [25]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'L3_GOES-E', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Meteosat Indian Ocean Sea Surface Temperature

OSI SAF distributes in **Near Real Time** data from <a href="https://osi-saf.eumetsat.int/products/osi-io-sst">Meteosat on Indian Ocean position</a>. As the two previous product, it is a hourly **level 3** product, aggregating the measurement made in an hour by th imager SEVIRI aboard Meteosat-9.<br>
Let's start by creating a specific folder to hold the Meteosat Indian Ocean Sea Surface Temperature.

In [26]:
download_dir = os.path.join(os.getcwd(), "products", "L3_Meteosat_IO")
os.makedirs(download_dir, exist_ok=True)

We set the download path where the datas are and we will chose a file in this path.

In [27]:
# Select date
Year, Month, Day, Hour = 2025, 7, 15, 10

In [28]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)
Hour_str = "{:02d}".format(Hour)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/indian/meteosat/' + str(Year) + '/' + day_of_year
filename = str(Year) + Month_str + Day_str + Hour_str + \
        '0000-OSISAF-L3C_GHRSST-SSTsubskin-SEVIRI_IO_SST-ssteqc_meteosat09_' + \
        str(Year) + Month_str + Day_str + '_' + Hour_str +'0000-v02.0-fv01.0.nc'
print(filename)

20250715100000-OSISAF-L3C_GHRSST-SSTsubskin-SEVIRI_IO_SST-ssteqc_meteosat09_20250715_100000-v02.0-fv01.0.nc


In [29]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'L3_Meteosat_IO', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### MSG/SEVIRI Sea Surface Temperature data record

OSI SAF produces and distribute as well a **Data Record** of the <a href="https://osi-saf.eumetsat.int/products/osi-250">Meteosat 0° data</a> from 2004 to 2012. It is a reprocessing of the level 1 data, having the same caracteristics as the **Near Real Time** products we downloaded earlier (hourly aggregate - Level 3). However, the overall quality of the data is better (but takes too long to produce for operational use). This data record currently based on **Meteosat-8** and **Meteosat-9** data, will be extended in the future to provide a wider temporal coverage.<br>
Let's start by creating a folder for these data.

In [30]:
download_dir = os.path.join(os.getcwd(), "products", "L3_Meteosat-0°_DataRecord")
os.makedirs(download_dir, exist_ok=True)

We set the dowload path where the datas are and we will chose a file in this path.

In [31]:
# Select date
Year, Month, Day, Hour =  2012, 12, 25, 0

In [32]:
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)
Hour_str = "{:02d}".format(Hour)

path = '/ifremer/cersat/projects/osisaf/sst/l3c/east_atlantic_west_indian/meteosat_data_record/v1/' + str(Year) + '/' + Month_str + '/' + Day_str
filename = str(Year) + Month_str + Day_str + Hour_str + \
        '0000-OSISAF-L3C_GHRSST-SSTsubskin-SEVIRI_SST-dr_ssteqc_meteosat09_' + \
        str(Year) + Month_str + Day_str + '_'+ Hour_str +'0000-v02.0-fv01.0.nc'

print(filename)

20121225000000-OSISAF-L3C_GHRSST-SSTsubskin-SEVIRI_SST-dr_ssteqc_meteosat09_20121225_000000-v02.0-fv01.0.nc


In [33]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'L3_Meteosat-0°_DataRecord', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

### Metop-B/IASI Sea Surface Temperature
OSI SAF distributes as well **level 2** data from polar orbiting satellites. One of these is the <a href="https://osi-saf.eumetsat.int/products/osi-208-b">IASI aboard Metop-B</a> SST measurement. The level 2 data are at full instrument resolution. Each netCDF correspond to 3min of measurement by the instrument called granules. One day of measurement will then consist in 480 files.<br>
Let's start by creating a specific folder to hold the data.

In [34]:
download_dir = os.path.join(os.getcwd(), "products", "L2_IASI")
os.makedirs(download_dir, exist_ok=True)

In [35]:
# Select date
Year, Month, Day = 2024, 1, 9

In [36]:
day_of_year = "{:03d}".format(datetime.datetime(Year, Month, Day).timetuple().tm_yday) # Calcul the day number of the year
Month_str = "{:02d}".format(Month)
Day_str = "{:02d}".format(Day)

path = '/ifremer/cersat/projects/osisaf/sst/l2p/global/iasi_metop_b/' + str(Year) + '/' + day_of_year

In [37]:
# connect ftp and download file
ftp = ftplib.FTP("ftp.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
file = ftp.nlst()
for filename in file:
    ftp.retrbinary("RETR " + filename, open(os.path.join(os.getcwd(), 'products', 'L2_IASI', filename), 'wb').write)
ftp.quit()

'221 Goodbye.'

After going through this notebook, you know how to basically access all the OSI SAF low and middle latitude SST data on the ifremer's FTP server. As you noticed however with IASI, downloading one day of Level 2 SST takes time and ressources. Moreover, we very often need SST on a particular area, and not globally. Ifremer is providing us with a tool to select the data depending on their localisation and area. That's what we are going to see in the <a href="./1_1b_OSI_SAF_SST_data_access_IFREMER_Opensearch.ipynb" target="_blank">next chapter</a>.

<hr>
<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1b_OSI_SAF_SST_data_access_IFREMER_Opensearch.ipynb" target="_blank">Accessing Metop SST granule data through IFREMER opensearch queries >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>